In [ ]:
install.packages("titanic")
library(titanic)
data(titanic_train)

head(titanic_train)
str(titanic_train)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<dbl>,<chr>,<chr>
1,1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.2500,,S
2,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
3,3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.9250,,S
4,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1000,C123,S
5,5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.0500,,S
6,6,0,3,"Moran, Mr. James",male,NA,0,0,330877,8.4583,,Q


'data.frame':	891 obs. of  12 variables:
 $ PassengerId: int  1 2 3 4 5 6 7 8 9 10 ...
 $ Survived   : int  0 1 1 1 0 0 0 0 1 1 ...
 $ Pclass     : int  3 1 3 1 3 3 1 3 3 2 ...
 $ Name       : chr  "Braund, Mr. Owen Harris" "Cumings, Mrs. John Bradley (Florence Briggs Thayer)" "Heikkinen, Miss. Laina" "Futrelle, Mrs. Jacques Heath (Lily May Peel)" ...
 $ Sex        : chr  "male" "female" "female" "female" ...
 $ Age        : num  22 38 26 35 35 NA 54 2 27 14 ...
 $ SibSp      : int  1 1 0 1 0 0 0 3 0 1 ...
 $ Parch      : int  0 0 0 0 0 0 0 1 2 0 ...
 $ Ticket     : chr  "A/5 21171" "PC 17599" "STON/O2. 3101282" "113803" ...
 $ Fare       : num  7.25 71.28 7.92 53.1 8.05 ...
 $ Cabin      : chr  "" "C85" "" "C123" ...
 $ Embarked   : chr  "S" "C" "S" "S" ...


In [ ]:
# Drop NA (missing values)
titanic_train <- na.omit(titanic_train)
nrow(titanic_train)

# Convert nominal data into factor
titanic_train$Survived <- factor(titanic_train$Survived)
titanic_train$Pclass <- factor(titanic_train$Pclass)
titanic_train$Sex <- factor(titanic_train$Sex)
titanic_train$Deck <- factor(ifelse(titanic_train$Cabin == "", "", substr(titanic_train$Cabin, 1, 1)))
titanic_train$Embarked <- factor(titanic_train$Embarked)

str(titanic_train)

[1] 714

'data.frame':	714 obs. of  13 variables:
 $ PassengerId: int  1 2 3 4 5 7 8 9 10 11 ...
 $ Survived   : Factor w/ 2 levels "0","1": 1 2 2 2 1 1 1 2 2 2 ...
 $ Pclass     : Factor w/ 3 levels "1","2","3": 3 1 3 1 3 1 3 3 2 3 ...
 $ Name       : chr  "Braund, Mr. Owen Harris" "Cumings, Mrs. John Bradley (Florence Briggs Thayer)" "Heikkinen, Miss. Laina" "Futrelle, Mrs. Jacques Heath (Lily May Peel)" ...
 $ Sex        : Factor w/ 2 levels "female","male": 2 1 1 1 2 2 2 1 1 1 ...
 $ Age        : num  22 38 26 35 35 54 2 27 14 4 ...
 $ SibSp      : int  1 1 0 1 0 0 3 0 1 1 ...
 $ Parch      : int  0 0 0 0 0 0 1 2 0 1 ...
 $ Ticket     : chr  "A/5 21171" "PC 17599" "STON/O2. 3101282" "113803" ...
 $ Fare       : num  7.25 71.28 7.92 53.1 8.05 ...
 $ Cabin      : chr  "" "C85" "" "C123" ...
 $ Embarked   : Factor w/ 4 levels "","C","Q","S": 4 2 4 4 4 4 4 4 2 4 ...
 $ Deck       : Factor w/ 9 levels "","A","B","C",..: 1 4 1 4 1 6 1 1 1 8 ...
 - attr(*, "na.action")= 'omit' Named int [1:177] 6 

In [ ]:
# Split Data
set.seed(42)
n <- nrow(titanic_train)
id <- sample(1:n, size=n*0.7) # 70% train 30% test
train_data <- titanic_train[id, ]
test_data <- titanic_train[-id, ]

# Train Data
model <- glm(Survived ~ Pclass + Sex + Age + SibSp, data = train_data, family="binomial")
summary(model)

p_train <- predict(model, type="response")
train_data$pred <- ifelse(p_train >= 0.5, "1", "0")
train_accuracy <- mean(train_data$Survived == train_data$pred)
cat("Train Accuracy:", train_accuracy, "\n") # 0.805611

# Test Data
p_test <- predict(model, newdata = test_data, type="response")
test_data$pred <- ifelse(p_test >= 0.5, "1", "0")
test_accuracy <- mean(test_data$Survived == test_data$pred)
cat("Test Accuracy:", test_accuracy, "\n") # 0.8


Call:
glm(formula = Survived ~ Pclass + Sex + Age + SibSp, family = "binomial", 
    data = train_data)

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  4.81542    0.57475   8.378  < 2e-16 ***
Pclass2     -1.67250    0.35348  -4.732 2.23e-06 ***
Pclass3     -3.01558    0.36235  -8.322  < 2e-16 ***
Sexmale     -2.83209    0.27051 -10.469  < 2e-16 ***
Age         -0.05071    0.01041  -4.870 1.12e-06 ***
SibSp       -0.41204    0.15549  -2.650  0.00805 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 673.56  on 498  degrees of freedom
Residual deviance: 424.41  on 493  degrees of freedom
AIC: 436.41

Number of Fisher Scoring iterations: 5


Train Accuracy: 0.8056112 
Test Accuracy: 0.8 
